In [1]:
import fs
import pandas as pd
import numpy as np

In [2]:
RAW_PROJECT_DIR = fs.open_fs("../../data/raw/project")
INTERIM_PROJECT_DIR = fs.open_fs("../../data/interim/project")
RAW_FILE_NAME = RAW_PROJECT_DIR.getsyspath("hmeq.csv")
INTERIM_FILE_NAME = INTERIM_PROJECT_DIR.getsyspath("use_to_divide.csv")

### Dataset Description

In [3]:
data = pd.read_csv(RAW_FILE_NAME)

data.shape

(5960, 13)

In [4]:
data.dtypes

BAD          int64
LOAN         int64
MORTDUE    float64
VALUE      float64
REASON      object
JOB         object
YOJ        float64
DEROG      float64
DELINQ     float64
CLAGE      float64
NINQ       float64
CLNO       float64
DEBTINC    float64
dtype: object

In [5]:
X = data.drop(columns=['BAD'])
y = data['BAD']

In [6]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categoric_features = X.select_dtypes(include=['object']).columns

In [7]:
data[numeric_features].describe().T

,count,mean,std,min,25%,50%,75%,max
LOAN,5960.0,18607.969799,11207.480417,1100.000000,11100.000000,16300.000000,23300.000000,89900.000000
MORTDUE,5442.0,73760.817200,44457.609458,2063.000000,46276.000000,65019.000000,91488.000000,399550.000000
VALUE,5848.0,101776.048741,57385.775334,8000.000000,66075.500000,89235.500000,119824.250000,855909.000000
YOJ,5445.0,8.922268,7.573982,0.000000,3.000000,7.000000,13.000000,41.000000
DEROG,5252.0,0.254570,0.846047,0.000000,0.000000,0.000000,0.000000,10.000000
DELINQ,5380.0,0.449442,1.127266,0.000000,0.000000,0.000000,0.000000,15.000000
CLAGE,5652.0,179.766275,85.810092,0.000000,115.116702,173.466667,231.562278,1168.233561
NINQ,5450.0,1.186055,1.728675,0.000000,0.000000,1.000000,2.000000,17.000000
CLNO,5738.0,21.296096,10.138933,0.000000,15.000000,20.000000,26.000000,71.000000
DEBTINC,4693.0,33.779915,8.601746,0.524499,29.140031,34.818262,39.003141,203.312149


In [8]:
X[categoric_features].describe().T

,count,unique,top,freq
REASON,5708,2,DebtCon,3928
JOB,5681,6,Other,2388


### Nulls

In [9]:
null_percentage = np.round(data.isnull().sum() * 100 / len(data), 2)

missing_values_df = pd.DataFrame({'missing_values_%': null_percentage})
missing_values_df

,missing_values_%
BAD,0.00
LOAN,0.00
MORTDUE,8.69
VALUE,1.88
REASON,4.23
JOB,4.68
YOJ,8.64
DEROG,11.88
DELINQ,9.73
CLAGE,5.17


### Duplicate Data

In [10]:
data.duplicated().sum()

0

### Imputing Nulls

In [11]:
data['JOB'] = data['JOB'].fillna('Other')

In [12]:
medians_job = data.groupby('JOB')[['MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']].median()
medians_job = medians_job.reset_index()
medians_job = medians_job.rename(columns={'MORTDUE': 'MED_MORTDUE', 'VALUE': 'MED_VALUE', 
'YOJ': 'MED_YOJ', 'DEROG': 'MED_DEROG', 'DELINQ': 'MED_DELINQ', 'CLAGE': 'MED_CLAGE', 
'NINQ': 'MED_NINQ', 'CLNO': 'MED_CLNO', 'DEBTINC': 'MED_DEBTINC'})
mode_reason = data.groupby('JOB')[['REASON']].agg(lambda x: x.mode().iloc[0]).reset_index()
mode_reason = mode_reason.rename(columns={'REASON': 'MED_MODREASON'})

In [13]:
mode_reason

,JOB,MED_MODREASON
0,Mgr,DebtCon
1,Office,DebtCon
2,Other,DebtCon
3,ProfExe,DebtCon
4,Sales,DebtCon
5,Self,HomeImp


In [14]:
medians_job.head()

,JOB,MED_MORTDUE,MED_VALUE,MED_YOJ,MED_DEROG,MED_DELINQ,MED_CLAGE,MED_NINQ,MED_CLNO,MED_DEBTINC
0,Mgr,76600.0,101258.0,8.0,0.0,0.0,171.214416,1.0,23.0,35.661118
1,Office,62970.5,89094.5,6.0,0.0,0.0,171.135632,1.0,20.0,36.158718
2,Other,55000.0,76864.5,7.0,0.0,0.0,156.204497,1.0,17.5,34.571519
3,ProfExe,82802.5,110007.0,8.0,0.0,0.0,185.170637,1.0,24.0,33.378041
4,Sales,71800.0,84473.5,4.0,0.0,0.0,147.355313,0.0,23.0,35.764058


In [15]:
data_to_get_okey = pd.merge(data, medians_job, on='JOB', how='left')
data_to_get_okey = data_to_get_okey.fillna({'MORTDUE': data_to_get_okey['MED_MORTDUE'], 
'VALUE': data_to_get_okey['MED_VALUE'], 
'YOJ': data_to_get_okey['MED_YOJ'], 
'DEROG': data_to_get_okey['MED_DEROG'], 
'DELINQ': data_to_get_okey['MED_DELINQ'], 
'CLAGE': data_to_get_okey['MED_CLAGE'],
'NINQ': data_to_get_okey['MED_NINQ'], 
'CLNO': data_to_get_okey['MED_CLNO'],
'DEBTINC': data_to_get_okey['MED_DEBTINC']}).drop(data_to_get_okey.filter(regex='^MED_').columns, axis=1)
data_okey =  pd.merge(data_to_get_okey, mode_reason, on='JOB', how='left')
data_okey = data_okey.fillna({'REASON': data_okey['MED_MODREASON']}).drop(['MED_MODREASON'], axis=1)

In [16]:
null_percentage = np.round(data_okey.isnull().sum() * 100 / len(data_okey), 2)

missing_values_df = pd.DataFrame({'missing_values_%': null_percentage})
missing_values_df

,missing_values_%
BAD,0.0
LOAN,0.0
MORTDUE,0.0
VALUE,0.0
REASON,0.0
JOB,0.0
YOJ,0.0
DEROG,0.0
DELINQ,0.0
CLAGE,0.0


In [17]:
data_okey

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,34.571519
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,34.571519
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,34.571519
3,1,1500,55000.0,76864.5,DebtCon,Other,7.0,0.0,0.0,156.204497,1.0,17.5,34.571519
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,36.158718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347
5956,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971
5957,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590
5958,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882


### Detecting Outliers

In [18]:
def find_outliers_iqr(df, columns=None):
    # If no specific columns are provided, use only numeric columns
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    
    # Create an empty list to store whether a row contains any outlier
    outlier_flags = []
    
    # Precompute IQR bounds for each numeric column
    iqr_bounds = {}
    for col in columns:
        Q1 = df[col].quantile(0.01)
        Q3 = df[col].quantile(0.99)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        iqr_bounds[col] = (lower_bound, upper_bound)
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        row_outlier = False  # Initialize flag for each row
        
        # Check each numeric column for an outlier using the precomputed bounds
        for col in columns:
            lower_bound, upper_bound = iqr_bounds[col]
            
            # Check if the current value in this row and column is an outlier
            if row[col] < lower_bound or row[col] > upper_bound:
                row_outlier = True
                break  # Stop checking other columns for this row once an outlier is found
        
        # Append 'Outlier' or 'Not Outlier' based on the row_outlier flag
        outlier_flags.append('Outlier' if row_outlier else 'Not Outlier')
    
    # Add a new column to the DataFrame
    df['outlier_flag'] = outlier_flags
    
    return df

In [19]:
outliers = find_outliers_iqr(data_okey)

In [20]:
outliers

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,outlier_flag
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,34.571519,Not Outlier
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,34.571519,Not Outlier
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,34.571519,Not Outlier
3,1,1500,55000.0,76864.5,DebtCon,Other,7.0,0.0,0.0,156.204497,1.0,17.5,34.571519,Not Outlier
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,36.158718,Not Outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0,90185.0,DebtCon,Other,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347,Not Outlier
5956,0,89000,54576.0,92937.0,DebtCon,Other,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971,Not Outlier
5957,0,89200,54045.0,92924.0,DebtCon,Other,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590,Not Outlier
5958,0,89800,50370.0,91861.0,DebtCon,Other,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882,Not Outlier


In [21]:
outliers.groupby('outlier_flag').size()

outlier_flag
Not Outlier    5947
Outlier          13
dtype: int64

### Save Clean Data

In [22]:
outliers.to_csv(INTERIM_FILE_NAME, index=False, encoding="utf-8")